## BIG QUESTIONS
best syntax?
how do deal with adding new parameters, should they be part of the sequence? would you ever add a parameter mid sequency?
there is parameter object and parameter analysisStep?

In [1]:
import functioneer as fn

In [11]:
### Choosing a Function to Analyze
# Functioneer is designed to analyze ANY function(s) with ANY number of inputs and outputs. For the following examples, the [Rosenbrock Function](https://en.wikipedia.org/wiki/Rosenbrock_function) is used for its relative simplicity, 4 inputs (plenty to play with) and its historical significance as an optimization benchmark.

# Rosenbrock function (known minimum of 0 at: x=1, y=1, a=1, b=100)
def rosenbrock(x, y, a, b):
    return (a-x)**2 + b*(y-x**2)**2

In [12]:
### Example 1: The Basics (Defining Parameters and Executing a Function)
# Set up an *analysis sequence* by defining four parameters (the inputs needed for the Rosenbrock function), then executing the function (with parameter ids matched to kwargs)

# Create new analysis
anal = fn.AnalysisModule() # its not ānal is anál!

# Define analysis sequence
anal.add(fn.Define('a', 1)) # Define parameter 'a'
anal.add(fn.Define('b', 100)) # Define parameter 'b'
anal.add(fn.Define('x', 1)) # Define parameter 'x'
anal.add(fn.Define('y', 1)) # Define parameter 'y'

anal.add(fn.Execute(func=rosenbrock, output_param_ids='rosen')) # Execute function with parameter ids matched to kwargs

# Run the analysis sequence
results = anal.run()

print(results['df'])

done with analysis!
    runtime  a    b  x  y  rosen                   datetime
0  0.001004  1  100  1  1      0 2025-01-09 13:51:09.003823


In [13]:
### Example 2: Single Parameter Forks (Testing Variations of a Parameter)
# If you want to test a set of values for a parameter you can create a *fork* in the *analysis sequence*. This splits the analysis into multiple *branches*, each exploring different values for a the given parameter.

# Say we want to evaluate and plot the Rosenbrock surface over the x-y domain. Let's evaluate Rosenbrock a grid where x=(0, 1, 2) and y=(1, 10) which should result in 6 final *branches* / *leaves*...

# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add(fn.Define('a', 1)) # Define parameter 'a'
anal.add(fn.Define('b', 100)) # Define parameter 'b'
anal.add(fn.Fork('x', value_sets=(0, 1, 2))) # Fork analysis, create a branch for each value of 'x': 0, 1, 2
anal.add(fn.Fork('y', value_sets=(1, 10))) # Fork analysis, create a branch for each value of 'y': 1, 10

anal.add(fn.Execute(func=rosenbrock, output_param_ids='rosen')) # Execute function (for each branch) with parameters matched to kwargs

# Run the analysis sequence
results = anal.run()

print(results['df'].drop(columns='datetime'))

done with analysis!
   runtime  a    b  x   y  rosen
0      0.0  1  100  0   1    101
1      0.0  1  100  0  10  10001
2      0.0  1  100  1   1      0
3      0.0  1  100  1  10   8100
4      0.0  1  100  2   1    901
5      0.0  1  100  2  10   3601


In [14]:
### Example 3: Optimization
# Lets say you want to find the local minimum of the Rosenbrock (optimize `x` and `y`) for several different flavors Rosenbrock functions (each with different `a` nnd `b` parameters). You would then fork the analysis at parameters `a` and `b` then after the forks perform the optimization on each branch.

# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add(fn.Fork('a', value_sets=(1, 2))) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add(fn.Fork('b', value_sets=(0, 100, 200))) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y')))

# Run the analysis sequence
results = anal.run()
print(results['df'].drop(columns='datetime'))

done with analysis!
    runtime  a    b         x         y         rosen
0  0.002002  1    0  1.000000  0.000000  4.930381e-32
1  0.012991  1  100  0.999763  0.999523  5.772481e-08
2  0.008002  1  200  0.999939  0.999873  8.146869e-09
3  0.000000  2    0  2.000000  0.000000  0.000000e+00
4  0.008009  2  100  1.999731  3.998866  4.067518e-07
5  0.010581  2  200  1.999554  3.998225  2.136755e-07


In [16]:
### Example 4: Multi-parameter Forks
# If you want to test specific combinations of parameters (instead of creating a grid) use a *multi-parameter fork*. The following will result in 3 branches: (a=0, b=0), (a=1, b=100), (a=2, b=200)



# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add(fn.Define('a', 1)) # Define parameter 'a'
anal.add(fn.Define('b', 100)) # Define parameter 'b'
anal.add(fn.Fork(('x', 'y'), value_sets=((0, 1, 2), (0, 10, 20)))) # Fork analysis, create a branch for each value of 'y': 1, 10

anal.add(fn.Execute(func=rosenbrock, output_param_ids='rosen')) # Execute function (for each branch) with parameters matched to kwargs

# Run the analysis sequence
results = anal.run()

print(results['df'].drop(columns='datetime'))

done with analysis!
   runtime  a    b  x   y  rosen
0      0.0  1  100  0   0      1
1      0.0  1  100  1  10   8100
2      0.0  1  100  2  20  25601


In [18]:
### Example 5: Analysis Steps can be Conditional
# Any *analysis step* can be given a conditional function that must return true at runtime or else the *analysis step* will be skipped. One use case for this is when you want to skip an expensive *analysis step* if the parameters aren't looking good.
# As an arbitrary example, assume that we only care about cases where the optimized value of `y` is above 0.5. Also assume `expensive_func` is costly to run and we want to avoid running it when `y<0.5`. 

# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add(fn.Fork('a', value_sets=(1, 2)))
anal.add(fn.Fork('b', value_sets=(0, 100, 200)))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))
anal.add(fn.Optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y')))

# Only evaluate 'expensive_func' if the optimized 'x' is above 0.5
expensive_func = lambda x, y: x+y
anal.add(fn.Execute(func=expensive_func, output_param_ids='expensive_param', condition=lambda y: y>0.5))

results = anal.run()
print(results['df'].drop(columns='datetime'))

done with analysis!
    runtime  a    b         x         y         rosen  expensive_param
0  0.002001  1    0  1.000000  0.000000  4.930381e-32              NaN
1  0.016232  1  100  0.999763  0.999523  5.772481e-08         1.999286
2  0.009002  1  200  0.999939  0.999873  8.146869e-09         1.999811
3  0.001003  2    0  2.000000  0.000000  0.000000e+00              NaN
4  0.010998  2  100  1.999731  3.998866  4.067518e-07         5.998596
5  0.011999  2  200  1.999554  3.998225  2.136755e-07         5.997779


In [5]:
# Different Output Modes Test

# test rosenbrock with various output formats
def rosenbrock_tuple(x, y, a, b):
    val = rosenbrock(x, y, a, b)
    val2 = 69
    return (val, val2)

def rosenbrock_dict(x, y, a, b):
    val = rosenbrock(x, y, a, b)
    val2 = 69
    return {'rosen_dict': val, 'val2_dict': val2}

# Create new analysis
anal = fn.AnalysisModule()

anal.add(fn.Define('a', 1))
anal.add(fn.Define('b', 100))
anal.add(fn.Define('x', 1))
anal.add(fn.Define('y', 1))

anal.add(fn.Execute(func=rosenbrock, output_param_ids='rosen')) # Direct output mode
anal.add(fn.Execute(func=rosenbrock_tuple, output_param_ids=('rosen_tuple', 'val2_tuple'))) # Positional output mode
anal.add(fn.Execute(func=rosenbrock_dict)) # Keyword output mode

results = anal.run()
results['df']


done with analysis!


,runtime,a,b,x,y,rosen,rosen_tuple,val2_tuple,rosen_dict,val2_dict,datetime
0,0.001002,1,100,1,1,0,0,69,0,69,2025-01-09 13:47:02.730670


In [6]:
# Multi Parameter Forks
# Create new analysis
anal = fn.AnalysisModule()
# Define analysis sequence
anal.add(fn.Define('a', 1)) # Deine parameter
anal.add(fn.Define('b', 100))
# anal.add(fn.Fork(('x', 'y'), value_sets=((0, 1, 2), (0, 1, 2)))) # Fork with new parameter
anal.add(fn.Fork('x', value_sets=((0, 1, 2), (0, 1, 2)))) # Fork with new parameter

results = anal.run()
results['df']


done with analysis!


,runtime,a,b,x,datetime
0,0.0,1,100,"(0, 1, 2)",2025-01-09 13:47:02.757668
1,0.0,1,100,"(0, 1, 2)",2025-01-09 13:47:02.758668


In [7]:
# Basic Optimization Test
anal = fn.AnalysisModule()

anal.add(fn.Define('a', 1))
anal.add(fn.Define('b', 100))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Fork('a', value_sets=(0, 1, 2)))
anal.add(fn.Fork('b', value_sets=(0, 100, 200)))
# anal.add(fn.Fork('b', value_sets=(0, 100, 200), condition=lambda a: a==1))

anal.add(fn.Optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y')))

results = anal.run()

print(f"{results['df']:.3f}")

done with analysis!


TypeError: unsupported format string passed to DataFrame.__format__

In [13]:
# Custom Optimizer Function
import scipy

anal = fn.AnalysisModule()

anal.add(fn.Define('a', 1))
anal.add(fn.Define('b', 100))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Fork('a', value_sets=(-1, 0, 1, 2)))

anal.add(fn.Optimize(obj_func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'), optimizer=scipy.optimize.minimize, optimizer_args=, optimizer_output=))
anal.add(fn.Optimize(
    optimizer_snippet = lambda paramset:(
        optimizer setup using paramset
    )
))

results = anal.run()

results['df']


## Version 2 ----------------------------------------------------------
# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add.fork('a', value_sets=(1, 2)) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add.fork('b', value_sets=(0, 100, 200)) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.add.define('x', 0)
anal.add.define('y', 0)

anal.add.fork.copy(num_copies=100)

anal.add.optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'))
anal.add.optimize.dict_return(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'), output_param_ids=('rosen2', 'rosen3'))
anal.add.optimize.custom(optimizer=scipy.optimize.minimize, x0_arg='x0', optimizer_args=dict(
    func=rosenbrock, 
)
(obj_func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'), optimizer=, , optimizer_output=)

anal.add.execute(output_param_id='expensive_param', func=expensive_func)
anal.add.execute.dict_return(func=expensive_func)
anal.add.execute.tuple_return(output_param_ids=('expensive_param', 'param2'), func=expensive_func)

# Run the analysis sequence
results = anal.run()



TypeError: Optimize.__init__() got an unexpected keyword argument 'optimizer'

In [ ]:


# optimizer mapping tool only

myfunc = lambda a, b, c, d: a+b+c+d
myfunc_kwargs = dict(a=1, b=2, c=3, d=4)

# wrapper function
myfunc_opt, myfunc_x0 = fn.optimize.kwarg_mapper(myfunc, def_kwargs=myfunc_kwargs, opt_vars=('a', 'b'))

# or use a decorator
@optimizer_ready(def_kwargs=myfunc_kwargs, opt_vars=('a', 'b'))
def myfunc_opt(a, b, c, d):
    return myfunc(a, b, c, d)

# or use a dynamic decorator
myfunc_opt = optimizer_ready(def_kwargs=myfunc_kwargs, opt_vars=('a', 'b'))(myfunc)


results = scipy.optimize.minimize(func=myfunc_opt, x0=myfunc_x0)


# what we are up against (custom mapping)
myfunc = lambda a, b, c, d: a+b+c+d

def myfunc_opt(x):
	a = x[0]
	b = x[1]
	c = 3
	d = 4
	return myfunc(a, b, c, d)

myfunc_x0 = [1, 2]

results = scipy.optimize.minimize(func=myfunc_opt, x0=myfunc_x0)



In [20]:
# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add(fn.Fork('a', value_sets=(1, 2)))
anal.add(fn.Fork('b', value_sets=(0, 100, 200)))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))
anal.add(fn.Optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y')))

# Only evaluate 'expensive_func' if the optimized 'x' is above 0.5
expensive_func = lambda x, y: x+y
anal.add(fn.Execute(func=expensive_func, output_param_ids='expensive_param', condition=lambda y: y>0.5))

results = anal.run()
print(results['df'])

done with analysis!
    runtime  a    b         x         y         rosen  \
0  0.004001  1    0  1.000000  0.000000  4.930381e-32   
1  0.009702  1  100  0.999763  0.999523  5.772481e-08   
2  0.017009  1  200  0.999939  0.999873  8.146869e-09   
3  0.000995  2    0  2.000000  0.000000  0.000000e+00   
4  0.016001  2  100  1.999731  3.998866  4.067518e-07   
5  0.020995  2  200  1.999554  3.998225  2.136755e-07   

                    datetime  expensive_param  
0 2025-01-08 00:38:07.088954              NaN  
1 2025-01-08 00:38:07.099666         1.999286  
2 2025-01-08 00:38:07.118677         1.999811  
3 2025-01-08 00:38:07.123193              NaN  
4 2025-01-08 00:38:07.141195         5.998596  
5 2025-01-08 00:38:07.166191         5.997779  


In [7]:
# Optimization Test with attached objective function
anal.add(fn.Define('a', 1))
anal.add(fn.Define('b', 100))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Fork('a', values=(-1, 0, 1, 2)))

anal.add(fn.Define('rosen', attached_func=rosenbrock, function_output_format=('direct', 'dict'), update_other_vars_when_run=True))

# Auto param mapping
anal.add(fn.Optimize('rosen'))

results = anal.run()

results['df']

TypeError: Fork.__init__() got an unexpected keyword argument 'values'

In [ ]:
# Parameter Reference
anal.add(fn.Define('a', 1))
anal.add(fn.Define('b', 100))
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Fork('a', values=(-1, 0, 1, 2)))

anal.add(fn.Execute(func=rosenbrock, output_param_ids='rosen')) # Direct output mode

anal.add(fn.Define('rosen', attached_func=rosenbrock, function_output_format=('direct', 'dict'), update_other_vars_when_run=True))

anal.add(fn.Fork('opt_method', values=('SLSQP', 'Nelder-Mead')))

anal.add(fn.Function(id='rosen', func=rosenbrock, output_param_ids='rosen_tuple'))


anal.add(fn.Optimize(objective=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'), method=fn.Reference('opt_method')))



In [ ]:
# Syntax Options
import functioneer as fn

## Version 1: Add steps as Objects ---------------------------------------------------------------------------------------
# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add(fn.Fork('a', value_sets=(1, 2))) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add(fn.Fork('b', value_sets=(0, 100, 200))) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.add(fn.Define('x', 0))
anal.add(fn.Define('y', 0))

anal.add(fn.Optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y')))

# Run the analysis sequence
results = anal.run()
print(results['df'].drop(columns='datetime'))


## Version 2: Add steps through the 'add' namespace ---------------------------------------------------------------------------------------
# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.add.fork('a', value_sets=(1, 2)) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.add.fork('b', value_sets=(0, 100, 200)) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.add.define('x', 0)
anal.add.define('y', 0)

anal.add.optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'))

# Run the analysis sequence
results = anal.run()


## Version 3: Add steps directly to Analysis ---------------------------------------------------------------------------------------
# Create new analysis
anal = fn.AnalysisModule()

# Define analysis sequence
anal.fork('a', value_sets=(1, 2)) # Fork analysis, create a branch for each value of 'a': 0, 1, 2
anal.fork('b', value_sets=(0, 100, 200)) # Fork analysis, create a branch for each value of 'b': 0, 100, 200
anal.define('x', 0)
anal.define('y', 0)

anal.optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y'))

# Run the analysis sequence
results = anal.run()


## Version 4: Declaritive list of Objects ---------------------------------------------------------------------------------------
# Create new analysis
anal = fn.AnalysisModule()

# Define the analysis as a configuration
anal = fn.AnalysisModule([
    fn.Fork('a', value_sets=(1, 2)),
    fn.Fork('b', value_sets=(0, 100, 200)),
    fn.Define('x', 0),
    fn.Define('y', 0),
    fn.Execute(method='default', func=rosenbrock),
    fn.Optimize(func=rosenbrock, obj_param_id='rosen', opt_param_ids=('x', 'y')),
])

# Run the analysis
results = anal.run()


In [ ]:
# pytorch example

# RNN Model
class MadagascarNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        """
        Initializes the MadagascarNN class (Ya Bassic Feed Forward NN) 
        
        Parameters:
        - input_dim (int): Dimensionality of the input features.
        - hidden_dims (list): List of integers, where each integer denotes the number of neurons
                              in a hidden layer. The length of the list determines the number of
                              hidden layers.
        - output_dim (int): Dimensionality of the output.
        """
        super(MadagascarNN, self).__init__()

        # Creating a list of all layers
        self.layers = nn.ModuleList()

        # Input to first hidden layer
        self.layers.append(nn.Linear(input_dim, hidden_dims[0]))
        self.layers.append(nn.ReLU())


In [2]:
from typing import Literal

def yo(x: Literal['up', 'down'], y: float):
    if x == 'up':
        return y
    return -y



In [3]:
yo(x='down', y=4)

-4